# Mnist Fashion Test

In [ ]:
import os
import urllib
import urllib.request
import cv2
import mlflow

import numpy as np
import matplotlib.pyplot as plt

from zipfile import ZipFile
import sys
sys.path.insert(1, r'../src')


import model

from utils import one_hot_encode_index

from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer
from loss import CategoricalCrossEntropyLoss

## Data Preparation 

In [ ]:
URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'fashion_mnist_images'

if not os.path.isfile(FILE):
    print(f'Downloading {URL} and saving as {FILE}...')
    urllib.request.urlretrieve(URL, FILE)

print('Unzipping images...')
with ZipFile(FILE) as zip_images:
    zip_images.extractall(FOLDER)

print('Done!')


## Loading Data 

In [ ]:
# Loads a MNIST dataset
def load_mnist_dataset(dataset, path):
    # Scan all the directories and create a list of labels
    labels = os.listdir(os.path.join(path, dataset))
    # Create lists for samples and labels
    X = []
    y = []
    # For each label folder
    for label in labels:
        # And for each image in given folder
        for file in os.listdir(os.path.join(path, dataset, label)):
            # Read the image
            image = cv2.imread(os.path.join(path, dataset, label, file), cv2.IMREAD_UNCHANGED)
            # And append it and a label to the lists
            X.append(image)
            y.append(label)
    # Convert the data to proper numpy arrays and return
    return np.array(X), np.array(y).astype('uint8')


def create_data_mnist(path):
    # Load both sets separately
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    y = one_hot_encode_index(y, 10)
    y_test = one_hot_encode_index(y_test, 10)
    
    # And return all the data
    return X, y, X_test, y_test


In [ ]:
X, y, X_test, y_test = create_data_mnist('fashion_mnist_images')


## Preprocess data

### Scaling between -1 & 1

In [ ]:
def scale_img(v):
    return (v - 127.5) /127.5

X = scale_img(X)
X_test = scale_img(X_test)

### Reshaping data 

In [ ]:
def vectorize(v):
    return v.reshape(v.shape[0], -1)

In [ ]:
X = vectorize(X)
X_test = vectorize(X_test)

### Shuffle Training data 

In [ ]:
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

## Training Model

### Setup MLfow 

In [ ]:
## TESTING 
import importlib
importlib.reload(model)
##

cce_loss = CategoricalCrossEntropyLoss()
optimizer = Adam(decay=5e-5)

my_model = model.Model(optimizer, cce_loss)

my_model.set_sequence([
                LinearLayer(X.shape[1], 128),
                ReLU(),
                Dropout(0.5),
                LinearLayer(128, 128),
                ReLU(),
                LinearLayer(128, 10),
                Softmax()
            ])


In [ ]:
mlflow.set_experiment(experiment_name='MNIST Fashion')

tags = {'Network_Type':'FFNN'}

with mlflow.start_run():
    my_model.train(X, y, epochs=10, batch_size=128, validation=(X_test, y_test))

In [ ]:
my_model.evaluate(X_test, y_test, 128)